# 2. Tensorflow

In [2]:
import os
import tempfile
import tensorflow as tf

from tensorflow.keras.datasets import imdb #imdb 데이터셋을 불러온다
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras import utils

import numpy as np

tf.logging.set_verbosity(tf.logging.INFO)
print(tf.__version__)

1.10.1


## 2.2 tf.data

  머신러닝에서 많은 시간을 소요하는 것은 데이터를 다루는 것이다. 일반적으로 데이터 분석, 전처리, 파이프라인 만드는 과정에서 약 70~80%의 시간을 소비한다 해도 과언이 아니다. 텐서플로우에서는 Dataset API를 활용하여, 단순한 연구자 뿐만 아니라 서비스화를 위해서 데이터에 대한 고민들을 해결 해 주고 있다. 학습 속도를 올리기 위해  GPU가 병목현상 없이 효율적으로 최적화를 지원하는 것 뿐만 아니라 이미지, 텍스트를 넘어 넘파이 (Numpy) 및 판다스(Pandas)데이터 구조가 활용하도록 지원이 가능하다.
  
  tf.data를 사용하기 위한 데이터를 활용하기 위해, IMDB 영화 리뷰 데이터셋을 불러오자. 데이터셋은 2만5천개의 리뷰의 데이터가 긍/부정으로 나누어져 있고, 전처리도 되어 있다. IMDB 영화 리뷰 데이터셋을 선정한 이유는 MNIST와 처럼 데이터셋을 tensorflow-keras를 활용하여 가장 손쉽게 불러오는 자연어 데이터이기 때문이다.

In [8]:
VOCAB_SIZE = 10000 #문장의 단어 사이즈
SENT_SIZE = 200 #문장 길이
EMB_SIZE = 128 #임베딩 사이즈
MODEL_DIR = tempfile.mkdtemp() #임시 디렉토리에 모델 저장

PAD_ID = 0 # 200단어 이하 문장에 대해서 0 값을 채워 넣는다.
START_ID = 1 # 시작 id 값
OOV_ID = 2 # out of vocab
INDEX_OFFSET = 2

#IMDB 데이터셋을 로드 합니다. 학습과 테스트 셋으로 나눕니다.
(xTrainVar, yTrain), (xTestVar, yTest) = imdb.load_data(num_words=VOCAB_SIZE,
                                                      start_char=START_ID,
                                                      oov_char=OOV_ID,
                                                      index_from=INDEX_OFFSET)

# 패딩 채워주기
print("Pad sequences (samples x time)")
xTrain = sequence.pad_sequences(xTrainVar, 
                                 maxlen=SENT_SIZE,
                                 truncating='post',
                                 padding='post',
                                 value=PAD_ID)
xTest = sequence.pad_sequences(xTestVar, 
                                maxlen=SENT_SIZE,
                                truncating='post',
                                padding='post', 
                                value=PAD_ID)

print("xTrain shape:", xTrain.shape)
print("xTest shape:", xTest.shape)

xLenTrain = np.array([min(len(x), SENT_SIZE) for x in xTrainVar])
xLenTest = np.array([min(len(x), SENT_SIZE) for x in xTest])

Pad sequences (samples x time)
xTrain shape: (25000, 200)
xTest shape: (25000, 200)
